In [2]:
import pathlib
import polars as pl
import plotly.express as px

pl.Config.set_fmt_float("full")

polars.config.Config

In [3]:
BASE_PATH = pathlib.Path.cwd().parent / "data" / "raw" / "parquet_files" / "train"

BASE_TRAIN_PATH = BASE_PATH / "train_base.parquet"

APPLPREV0_PATH = BASE_PATH / "train_applprev_1_0.parquet"
APPLPREV1_PATH = BASE_PATH / "train_applprev_1_1.parquet"

DEBITCARD_PATH = BASE_PATH / "train_debitcard_1.parquet"
DEPOSIT_PATH = BASE_PATH / "train_deposit_1.parquet"
OTHER_PATH = BASE_PATH / "train_other_1.parquet"
PERSON_PATH = BASE_PATH / "train_person_1.parquet"

## Functions

In [4]:
def get_date_categorized_transforms(dataframe: pl.LazyFrame) -> list[pl.Expr]:
    date_transforms = []
    
    for col in dataframe.columns:
        if not col.endswith("D"):
            continue

        transform = (
            pl.when(pl.col(col) > 365 * 5).then(pl.lit("MORE_THAN_5_YEARS")).otherwise(
                pl.when((pl.col(col) <= 365 * 5) & (pl.col(col) > 365)).then(pl.lit("BETWEEN_1_AND_5_YEARS")).otherwise(
                    pl.when((pl.col(col) <= 365) & (pl.col(col) > 180)).then(pl.lit("BETWEEN_6_MONTHS_AND_1_YEAR")).otherwise(
                        pl.when((pl.col(col) <= 180) & (pl.col(col).is_not_null())).then(pl.lit("LESS_THAN_6_MONTH")).otherwise(pl.lit("UNKNOWN"))
                    )
                )
            ).alias(col))
        date_transforms.append(transform)
        
    return date_transforms


def get_date_transforms(dataframe: pl.LazyFrame) -> list[pl.Expr]:
    date_transforms = []
    for col in dataframe.columns:
        if col == "date_decision":
            date_transforms.append(pl.col(col).cast(pl.Date))
            date_transforms.extend([
                ((pl.col("date_decision").cast(pl.Date) - pl.col("birth_259D").cast(pl.Date)).dt.total_days() // 365).alias("age"),
                ((pl.col("date_decision").cast(pl.Date) - pl.col("birth_259D").cast(pl.Date)).dt.total_days() % 365).sin().alias("sin(age_days)"),
                ((pl.col("date_decision").cast(pl.Date) - pl.col("birth_259D").cast(pl.Date)).dt.total_days() % 365).cos().alias("cos(age_days)")
            ])
        
        if not col.endswith("D"):
            continue
        
        date_transforms.append(
            (pl.col("date_decision").cast(pl.Date) - pl.col(col).cast(pl.Date)).dt.total_days().alias(col)
        )
        
    return date_transforms


def get_group_by_transforms(dataframe: pl.LazyFrame, group_by_col: str) -> list[pl.Expr]:
    transforms = []
    for col, col_dtype in zip(dataframe.columns, dataframe.dtypes):
        if col == group_by_col:
            continue
        
        if isinstance(col_dtype, tuple(pl.NUMERIC_DTYPES)):
            transforms.append(pl.col(col).mean().shrink_dtype())
            
        elif isinstance(col_dtype, pl.String):
            transforms.append(pl.col(col).mode().drop_nulls().first())

        elif isinstance(col_dtype, pl.Date):
            transforms.append(pl.col(col).first())
        
    return transforms


def get_columns_with_more_85_perc_nulls(dataframe: pl.LazyFrame, threshold: float = 0.85) -> list[str]:
    dataframe = dataframe.collect()
    length_dataframe: int = dataframe.shape[0]

    describe = dataframe.describe(percentiles=[]).filter(pl.col("statistic") == "null_count")
    describe = describe.transpose(include_header=True, header_name="column", column_names=["null_count"]).slice(1).with_columns(pl.col("null_count").cast(pl.Float32) / length_dataframe)
    
    return describe.filter(pl.col("null_count") > threshold).select("column").to_series().to_list()
    
    
def binarize_columns_with_more_85_perc_nulls(columns: list[str]) -> list[pl.Expr]:
    transforms = []
    for col in columns:
        transforms.append(
            pl.when(pl.col(col).is_null()).then(pl.lit(False)).otherwise(pl.lit(True)).alias(col)
        )
    return transforms


## Load data

In [5]:
JOIN_COLS = ["case_id", "num_group1"]

In [6]:
base_train: pl.LazyFrame = pl.scan_parquet(BASE_TRAIN_PATH).select(["case_id", "date_decision", "target"])
applprev_data: pl.LazyFrame = pl.concat([pl.scan_parquet(APPLPREV0_PATH), pl.scan_parquet(APPLPREV1_PATH)]).drop(["childnum_21L", "education_1138M", "familystate_726L"])
# debitcard_data: pl.LazyFrame = pl.scan_parquet(DEBITCARD_PATH)
deposit_data: pl.LazyFrame = pl.scan_parquet(DEPOSIT_PATH)
other_data: pl.LazyFrame = pl.scan_parquet(OTHER_PATH)
person_data: pl.LazyFrame = pl.scan_parquet(PERSON_PATH).select([
    "case_id",
    "num_group1",
    "birth_259D",
    "incometype_1044T",
    "role_1084L",
    "empl_employedfrom_271D",
    "empl_industry_691L",
    "mainoccupationinc_384A",
])

data: pl.LazyFrame = (applprev_data
        .join(deposit_data, how="left", on=JOIN_COLS)
        .join(other_data, how="left", on=JOIN_COLS)
        .join(person_data, how="left", on=JOIN_COLS))

data = base_train.select("case_id", "date_decision").join(data, how="left", on="case_id")

data.sort(JOIN_COLS).head().collect()

case_id,date_decision,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,employedfrom_700D,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L,amount_416A,contractenddate_991D,openingdate_313D,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,birth_259D,incometype_1044T,role_1084L,empl_employedfrom_271D,empl_industry_691L,mainoccupationinc_384A
i64,str,f64,f64,str,f64,str,str,f64,f64,f64,f64,str,f64,f64,str,f64,str,str,f64,str,str,str,str,str,bool,bool,f64,f64,i64,f64,f64,str,str,str,str,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,f64
0,"""2019-01-03""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,"""2019-01-03""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,"""2019-01-04""",0,640.2,null,null,"""a55475b1""","""2013-04-03""",null,0,null,null,null,null,10000,"""CAL""",null,null,"""P136_108_173""",0,null,null,"""2010-02-15""","""2013-05-04""","""CASH""",false,null,8200,null,0,null,24,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",24,null,null,null,null,null,null,null,null,"""1974-12-01""","""EMPLOYED""","""EM""","""2010-02-15""","""OTHER""",14000
2,"""2019-01-04""",0,1682.4,null,null,"""a55475b1""","""2013-04-03""",null,0,null,null,null,null,16000,"""CAL""",null,null,"""P136_108_173""",0,null,null,"""2010-02-15""","""2013-05-04""","""CASH""",false,null,8200,null,1,null,12,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""D""",12,null,null,null,null,null,null,null,null,null,null,"""CL""",null,null,null
3,"""2019-01-03""",0,6140,null,null,"""P94_109_143""","""2019-01-07""",null,0,null,null,null,null,59999.8,"""CAL""",null,null,"""P131_33_167""",0,null,null,"""2018-05-15""","""2019-02-07""","""CASH""",false,null,11000,null,0,null,12,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",null,"""D""",12,null,null,null,null,null,null,null,null,"""1993-08-01""","""EMPLOYED""","""CL""","""2018-05-15""","""OTHER""",10000


In [7]:
cols_to_binarize = get_columns_with_more_85_perc_nulls(data)

data = data.with_columns(*binarize_columns_with_more_85_perc_nulls(cols_to_binarize))

In [8]:
data = data.with_columns(*get_date_transforms(data)).drop("birth_259D")
data = data.with_columns(*get_date_categorized_transforms(data))
data = data.with_columns(pl.col(pl.Boolean).fill_null(False).cast(pl.Int8))
data = data.with_columns(
    # pl.col("target").cast(pl.Boolean),
    pl.col("age").cast(pl.Int16),
    pl.col("num_group1").cast(pl.Int16),
)

data.sort("case_id").head().collect()

case_id,date_decision,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,employedfrom_700D,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L,amount_416A,contractenddate_991D,openingdate_313D,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,incometype_1044T,role_1084L,empl_employedfrom_271D,empl_industry_691L,mainoccupationinc_384A,age,sin(age_days),cos(age_days)
i64,date,f64,f64,str,f64,str,str,i8,f64,i8,i8,i8,i8,f64,str,f64,str,str,f64,str,str,str,str,str,i8,i8,f64,f64,i16,f64,f64,str,str,str,str,i8,str,f64,i8,str,str,i8,i8,i8,i8,i8,str,str,str,i8,f64,i16,f64,f64
0,2019-01-03,null,null,"""UNKNOWN""",null,null,"""UNKNOWN""",0,null,0,0,0,0,null,null,null,"""UNKNOWN""",null,null,"""UNKNOWN""","""UNKNOWN""","""UNKNOWN""","""UNKNOWN""",null,0,0,null,null,null,null,null,null,null,null,null,0,null,null,0,"""MORE_THAN_5_YE…","""MORE_THAN_5_YE…",0,0,0,0,0,null,null,"""MORE_THAN_5_YE…",0,null,null,null,null
1,2019-01-03,null,null,"""UNKNOWN""",null,null,"""UNKNOWN""",0,null,0,0,0,0,null,null,null,"""UNKNOWN""",null,null,"""UNKNOWN""","""UNKNOWN""","""UNKNOWN""","""UNKNOWN""",null,0,0,null,null,null,null,null,null,null,null,null,0,null,null,0,"""MORE_THAN_5_YE…","""MORE_THAN_5_YE…",0,0,0,0,0,null,null,"""MORE_THAN_5_YE…",0,null,null,null,null
2,2019-01-04,0,640.2,"""UNKNOWN""",null,"""a55475b1""","""MORE_THAN_5_YE…",0,0,0,0,0,0,10000,"""CAL""",null,"""UNKNOWN""","""P136_108_173""",0,"""UNKNOWN""","""UNKNOWN""","""MORE_THAN_5_YE…","""MORE_THAN_5_YE…","""CASH""",0,0,8200,null,0,null,24,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0,"""D""",24,0,"""MORE_THAN_5_YE…","""MORE_THAN_5_YE…",0,0,0,0,0,"""EMPLOYED""","""EM""","""MORE_THAN_5_YE…",1,14000,44,0.8509035245341184,0.5253219888177297
2,2019-01-04,0,1682.4,"""UNKNOWN""",null,"""a55475b1""","""MORE_THAN_5_YE…",0,0,0,0,0,0,16000,"""CAL""",null,"""UNKNOWN""","""P136_108_173""",0,"""UNKNOWN""","""UNKNOWN""","""MORE_THAN_5_YE…","""MORE_THAN_5_YE…","""CASH""",0,0,8200,null,1,null,12,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0,"""D""",12,0,"""MORE_THAN_5_YE…","""MORE_THAN_5_YE…",0,0,0,0,0,null,"""CL""","""MORE_THAN_5_YE…",0,null,null,null,null
3,2019-01-03,0,6140,"""UNKNOWN""",null,"""P94_109_143""","""LESS_THAN_6_MO…",0,0,0,0,0,0,59999.8,"""CAL""",null,"""UNKNOWN""","""P131_33_167""",0,"""UNKNOWN""","""UNKNOWN""","""BETWEEN_6_MONT…","""LESS_THAN_6_MO…","""CASH""",0,0,11000,null,0,null,12,"""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""",0,"""D""",12,0,"""MORE_THAN_5_YE…","""MORE_THAN_5_YE…",0,0,0,0,0,"""EMPLOYED""","""CL""","""MORE_THAN_5_YE…",1,10000,25,-0.702407785577371,-0.7117747556357236


In [34]:
describe: dict[str, list[str | float]] = data.select("case_id").describe(percentiles=[x/10 for x in range(10)]).filter((pl.col("statistic").str.contains("%")) | (pl.col("statistic").str.contains("max"))).to_dict(as_series=False)
case_id_percentiles = sorted(describe.get("case_id"))

for i in range(len(case_id_percentiles) - 1):
    print(case_id_percentiles[i], case_id_percentiles[i+1])    
    slice_data: pl.LazyFrame = data.filter((pl.col("case_id") >= case_id_percentiles[i]) & (pl.col("case_id") < case_id_percentiles[i+1]))
    slice_data.group_by("case_id").agg(*get_group_by_transforms(data, "case_id")).collect().write_parquet(f"../data/bronze/depth_1/slice_{i}.parquet")

# merge files and delete slices
pl.scan_parquet(f"../data/bronze/depth_1/slice_*.parquet").collect().write_parquet(f"../data/bronze/depth_1.parquet")
for path in pathlib.Path("../data/bronze/depth_1").glob("slice*"):
    path.unlink(missing_ok=True)

0.0 178735.0
178735.0 255642.0
255642.0 1246716.0
1246716.0 1388752.0
1388752.0 1526194.0
1526194.0 1657782.0
1657782.0 1785216.0
1785216.0 1896140.0
1896140.0 2591656.0
2591656.0 2703454.0
